In [1]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_monthly_close_prices(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('BM', how=lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [34]:
quandl.get_table('SHARADAR/SEP',date={'gte':'2018-12-25','lte':'2019-01-01'}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0).head()

,ticker,date,close
None,,,
0,ZYXI,2018-12-31,2.94
1,ZYXI,2018-12-28,2.65
2,ZYXI,2018-12-27,2.63
3,ZYXI,2018-12-26,2.63
4,ZYNE,2018-12-31,2.97


In [3]:
monthly_prices = get_monthly_close_prices('2018-06-01','2019-01-01')

/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """


In [4]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [5]:
monthly_retuns_june_dec_18 = get_monthly_returns(monthly_prices)


In [6]:
monthly_retuns_june_dec_18.head()

ticker,A,AA,AAC,AAGIY,AAL,AALBF,AAMC,AAME,AAN,AAOI,...,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZURVY,ZX,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2018-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31,0.067917,-0.077005,0.108858,0.003711,0.041623,NaN,-0.060215,-0.037037,-0.003222,-0.144098,...,0.024510,-0.008500,0.015143,-0.095808,-0.098162,0.039634,NaN,-0.161705,-0.317248,-0.035484
2018-08-31,0.022714,0.032355,-0.147257,-0.019909,0.023773,NaN,-0.053394,0.057692,0.148003,0.076243,...,-0.002392,-0.069087,0.047641,0.375276,0.124338,-0.008472,NaN,0.133979,0.207519,0.030100
2018-09-28,0.044418,-0.095590,-0.138826,0.033082,0.020998,-0.055386,-0.035455,-0.090909,0.095334,-0.403772,...,-0.019185,-0.102384,0.010596,-0.154093,-0.162074,0.034834,NaN,0.117438,0.016189,-0.032468
2018-10-31,-0.081514,-0.133911,-0.277851,-0.152528,-0.151222,0.000000,-0.181287,0.120000,-0.134594,-0.203974,...,-0.171149,-0.021123,-0.015400,-0.117268,-0.116400,-0.016196,NaN,-0.186624,-0.355392,0.030201


In [7]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.iloc[ 1: , : ]

In [8]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.fillna(0)

In [9]:
#training_returns = monthly_retuns_june_dec_18.iloc[ :5 , : ]

In [10]:
#target_returns = monthly_retuns_june_dec_18.iloc[ 5:6 , : ]

In [11]:
def get_factor_data(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [62]:
#quandl.get_table('SHARADAR/SF1',dimension='ARY',calendardate={'gte':'2018-06-01', 'lte':'2019-01-01'}, paginate=True)

In [12]:
factors_june_dec_18 = get_factor_data('2018-06-01','2019-01-01')

In [13]:
factors_june_dec_18.head

<bound method NDFrame.head of       ticker dimension calendardate    datekey reportperiod lastupdated  \
None                                                                      
0       ZYXI       ARQ   2018-12-31 2019-02-26   2018-12-31  2019-02-26   
1       ZYXI       ARQ   2018-09-30 2018-11-06   2018-09-30  2019-02-26   
2       ZYXI       ARQ   2018-06-30 2018-08-02   2018-06-30  2019-02-26   
3       ZYNE       ARQ   2018-12-31 2019-03-11   2018-12-31  2019-03-11   
4       ZYNE       ARQ   2018-09-30 2018-11-08   2018-09-30  2019-03-11   
5       ZYNE       ARQ   2018-06-30 2018-08-02   2018-06-30  2019-03-11   
6       ZYME       ARQ   2018-12-31 2019-03-06   2018-12-31  2019-03-06   
7       ZYME       ARQ   2018-09-30 2018-11-06   2018-09-30  2019-03-06   
8       ZYME       ARQ   2018-06-30 2018-08-01   2018-06-30  2019-03-06   
9        ZUO       ARQ   2018-09-30 2018-12-13   2018-10-31  2018-12-13   
10       ZUO       ARQ   2018-06-30 2018-09-12   2018-07-31  2018-12-1

In [14]:
factors_june_dec_18.shape

(12658, 111)

In [18]:
df = get_factor_data('2007-12-31','2019-01-01')


In [19]:
corr_matrix = df.corr().abs()

In [21]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [22]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [23]:
factors_june_dec_18_reduced = factors_june_dec_18.drop(factors_june_dec_18[to_drop], axis=1)

In [24]:
factors_june_dec_18_reduced.

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sbcomp,sgna,sharefactor,sharesbas,shareswadil,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,ZYXI,ARQ,2018-12-31,2019-02-26,2018-12-31,2019-02-26,0,16184000.0,0,14326000.0,...,370000.0,4626000.0,1.0,33309244.0,33659000.0,725000.0,257000.0,688000,0.501,7971000.0
1,ZYXI,ARQ,2018-09-30,2018-11-06,2018-09-30,2019-02-26,0,13297000.0,0,12154000.0,...,0.0,3670000.0,1.0,32276614.0,33931000.0,0.0,228000.0,51000,0.409,8632000.0
2,ZYXI,ARQ,2018-06-30,2018-08-02,2018-06-30,2019-02-26,0,11393000.0,0,10245000.0,...,0.0,3528000.0,1.0,32673230.0,34169000.0,0.0,260000.0,129000,0.349,6978000.0
3,ZYNE,ARQ,2018-12-31,2019-03-11,2018-12-31,2019-03-11,0,67327443.0,0,66955480.0,...,1598630.0,3256044.0,1.0,21069996.0,17641728.0,3444620.0,0.0,0,3.816,57229698.0
4,ZYNE,ARQ,2018-09-30,2018-11-08,2018-09-30,2019-03-11,0,72934341.0,0,72685600.0,...,1584230.0,3125780.0,1.0,17626873.0,16587353.0,3095195.0,0.0,0,4.397,63544533.0


In [31]:
factors_june_dec_18_reduced = factors_june_dec_18_reduced.insert[]

SyntaxError: invalid syntax (<ipython-input-31-45515ec7ccef>, line 1)

In [ ]:
pivot(index='date', columns='ticker', values='close')

In [36]:
monthly_retuns_june_dec_18.pivot(columns=['ticker','date','close'])

ValueError: all arrays must be same length

In [27]:
df.head()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,ZZ,ARQ,2012-03-31,2012-03-27,2012-02-26,2018-05-16,5986000.0,936259000.0,0,359986000.0,...,100971540.0,100918000.0,109254000.0,12.257,572536000.0,24393000.0,2527000.0,554000.0,5.673,192297000.0
1,ZZ,ARQ,2011-12-31,2012-01-18,2011-11-27,2018-05-16,-509000.0,919194000.0,0,342095000.0,...,100916228.0,99261000.0,99261000.0,12.393,557052000.0,23121000.0,-3675000.0,549000.0,5.612,185598000.0
2,ZZ,ARQ,2011-09-30,2011-09-27,2011-08-28,2018-05-16,10703000.0,947852000.0,0,363128000.0,...,100820532.0,100334000.0,308566000.0,11.741,583322000.0,24183000.0,9556000.0,823000.0,5.814,194752000.0
3,ZZ,ARQ,2011-06-30,2011-06-28,2011-05-29,2018-05-16,11824000.0,932652000.0,0,343099000.0,...,98184056.0,98040000.0,107933000.0,12.139,567901000.0,25295000.0,-568000.0,842000.0,5.793,176708000.0
4,ZZ,ARQ,2011-03-31,2011-03-29,2011-02-27,2018-05-16,10733000.0,949099000.0,0,357615000.0,...,98120413.0,97816000.0,107828000.0,12.118,584330000.0,25374000.0,-1209000.0,843000.0,5.974,177924000.0


In [ ]:



features = factors_june_dec_18_reduced
target = monthly_retuns_june_dec_18

In [ ]:
features.rename(columns={"calendardate": "date"})

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)